# Sample Code to Query Data from and write to a Google Sheet
<br/><br/>
<br/><br/>
## Query Data from a Google Sheet
### Step 1: Share Google Sheet with the EMR service account
Go to the google sheet you want to query data from and share it with the OCFO EMR service account: emr-gsheet-api@prj-p-bia-emr-uatgsheet.iam.gserviceaccount.com
<br/><br/>
<br/><br/>
*Code modified from a script from https://github.helix.gsa.gov/EDA/Sample_Code* 
_______

In [19]:
import datetime

print(datetime.datetime.now())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2024-06-25 12:25:13.530469

In [20]:
%%configure -f
{
    "conf": {
        "spark.pyspark.python": "python3",
        "spark.pyspark.virtualenv.enabled": "true",
        "spark.pyspark.virtualenv.type":"native",
        "spark.pyspark.virtualenv.bin.path":"/usr/bin/virtualenv"
    }
}

Starting Spark application


ID,Kind,State,Spark UI,Driver log,User,Current session?
2,pyspark,idle,Link,,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,Kind,State,Spark UI,Driver log,User,Current session?
2,pyspark,idle,Link,,None,✔


### Step 2: Import needed packages

In [21]:
# install necessary packages
sc.install_pypi_package("google-api-python-client")
sc.install_pypi_package("google-auth-oauthlib")
sc.install_pypi_package("google-auth-httplib2")
# import packages for google api and authentication and data reading pandas
from googleapiclient.discovery import build
from google.oauth2 import service_account
import pandas as pd

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<br/><br/>
<br/><br/>
____
### Step 3: Input needed parameters
- Using example: 
https://docs.google.com/spreadsheets/d/1G3YXcXBC2J6AFOpxYg2WfSsjtg0p4xAgd5WEy2TSb2k/edit?gid=0#gid=0

In [22]:
# link to google sheet used in this example https://docs.google.com/spreadsheets/d/1G3YXcXBC2J6AFOpxYg2WfSsjtg0p4xAgd5WEy2TSb2k/edit?gid=0#gid=0
## it is revenue projection errors from cbo evals output
SPREADSHEET_ID = "1G3YXcXBC2J6AFOpxYg2WfSsjtg0p4xAgd5WEy2TSb2k"

RANGE_NAME = "Sheet1!A1:M14" 
CHANGE_RANGE_NAME = "Sheet1!A15"


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
#Gather Service Account Credentials


def read_service_account_credentials(SERVICE_ACCOUNT_FILE):
    """This function reads in the service account credentials (the json from Google Developer) from S3 and returns credentials in a dictionary
        SERVICE_ACCOUNT_FILE (str): the S3 path of json keys
    """
    file = spark.read.option("multiline","true").json(SERVICE_ACCOUNT_FILE)
    json_map = map(lambda row: row.asDict(), file.collect())
    for a in json_map:
         json_dict = a
            
    return json_dict
        
SERVICE_ACCOUNT_FILE = "s3://712311343692-bsp-lde-external/OCFO/PRD/IZZY_METZGER/prj-p-bia-emr-uatgsheet-bbd21d4dbfe0-keys.json"

json_dict_credentials = read_service_account_credentials(SERVICE_ACCOUNT_FILE)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<br/><br/>
<br/><br/>
____
### Step 4: Define queryGoogleSheet function

In [24]:
from google.oauth2 import service_account

def queryGoogleSheet(SPREADSHEET_ID, RANGE_NAME, json_dict_credentials):
    '''The queryGoogleSheet function, as the name implies, queries data from a specific range in a google sheet
       
       It takes three params:
            SPREADSHEET_ID (str): the id of the google sheet you want to query that you have already shared with the service account
            RANGE_NAME (str): The range where the data you want to query lives
            json_dict_credentials : google api key credentials
            
       returns PySpark DataFrame of the query'''
    

    creds = service_account.Credentials.from_service_account_info(
        json_dict_credentials)
    
    # Call the Sheets API
    service = build('sheets', 'v4', credentials=creds)
    sheet = service.spreadsheets()
    result = sheet.values().get(spreadsheetId=SPREADSHEET_ID, range=RANGE_NAME).execute()

    values = result.get('values', [])

    df  = pd.DataFrame(values, columns = values[0]).drop(0)
    outDF= spark.createDataFrame(df)
    return outDF

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<br/><br/>
<br/><br/>
____
## Step 5: Call queryGoogleSheet function

In [25]:
Results = queryGoogleSheet(SPREADSHEET_ID, RANGE_NAME, json_dict_credentials)
Results.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+--------+-----------+-------------+-----------+-----------+---------------------+---------------------+--------+------------+--------+--------------------------+-------------------+
|component|category|subcategory|baseline_date|Spring_flag|Winter_flag|projected_fiscal_year|projected_year_number|   value|actual_value|     GDP|legislative_revenue_change|adjusted_projection|
+---------+--------+-----------+-------------+-----------+-----------+---------------------+---------------------+--------+------------+--------+--------------------------+-------------------+
|  revenue|   Total|      Total|   1982-02-01|      FALSE|       TRUE|                 1982|                    1|     631|     617.766| 3313.35|                         0|                631|
|  revenue|   Total|      Total|   1983-02-01|      FALSE|       TRUE|                 1983|                    1|     606|     600.562|    3536|                         0|                606|
|  revenue|   Total|      Total|   

<br/><br/>
<br/><br/>
____
## Write Data to a google sheet

In [26]:
creds = service_account.Credentials.from_service_account_info(
        json_dict_credentials)
    
    # Call the Sheets API
service = build('sheets', 'v4', credentials=creds)

update_data = [["This",	"is", "an", "updated", "row", "1"]]
sheet = service.spreadsheets()
request = sheet.values().update(spreadsheetId=SPREADSHEET_ID, 
                                range=CHANGE_RANGE_NAME, 
                                valueInputOption="RAW", 
                                body={"values":update_data}).execute()



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
import datetime
print(datetime.datetime.now())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2024-06-25 12:26:57.983921

<br/><br/>
<br/><br/>
____
## Appendix: Steps to activate API
**Note: steps are here for you to reference**

Youtube help link (start min 3:30): https://www.youtube.com/watch?v=4ssigWmExak&t=10s

- Create ticket in service now so that GSA Google team can createa project in Developers console: https://console.cloud.google.com/apis/dashboard (cannot seem to use gsa email to create project.. I used a different email to create project and i can still pull gsa google sheets)
- request for google sheets api to be enabled
- Search google sheets api and enable - they will Create credentials in google sheets api
- copy this email and go to google sheets page you want to pull and share google sheets to service account email and make it an editor
-  click on your service account in google developers console and add key -> create new key as json
- Download your key and upload to location in s3
- change SERVICE_ACCOUNT_FILE below to new s3 location of json keys
- change SAMPLE_RANGE_NAME to range of google sheet you want to change
- change SAMPLE_SPREADSHEET_ID to google sheet id which is located in the google sheet url EX:https://docs.google.com/spreadsheets/d/1kxqidEneTbs4WF8WCg1dSJjcOgL77YXlbZFRwVWfnSs/edit#gid=0
id = 1kxqidEneTbs4WF8WCg1dSJjcOgL77YXlbZFRwVWfnSs
- run this code below to read in google sheet and then write out to google sheet 